# 12.4 演習




리스트12.13 각 설정

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam

optimizer = Adam()

img_size = 64  # 이미지의 폭과 높이
n_channel = 3  # 채널 수
n_mid = 256  # 중간층의 뉴런 수

batch_size = 32
epochs = 20

리스트12.14 VGG16의 도입

In [ ]:
from tensorflow.keras.applications import VGG16

model_vgg16 = VGG16(weights="imagenet",  # ImageNet에서 학습한 파라미터를 사용
                 include_top=False,  # 전결합층을 포함하지 않는다
                 input_shape=(img_size, img_size, n_channel))  # 입력의 형태
model_vgg16.summary()

리스트12.15 CIFAR-10 읽어 들이기


In [ ]:
from tensorflow.keras.datasets import cifar10

(x_train, t_train), (x_test, t_test) = cifar10.load_data()

# 인덱스가 6과 8의 데이터만 꺼낸다
cifar10_labels = np.array(["airplane", "automobile", "bird", "cat", "deer",
                           "dog", "frog", "horse", "ship", "truck"])
t1, t2 = 6, 8  # frogとship
t_train = t_train.reshape(-1)
t_test = t_test.reshape(-1)
mask_train = np.logical_or(t_train==t1, t_train==t2)  # 6과 と8만 True
x_train = x_train[mask_train]
t_train = t_train[mask_train]
mask_test = np.logical_or(t_test==t1, t_test==t2)  # 6과 8만True
x_test = x_test[mask_test]
t_test = t_test[mask_test]

# 이미지를 확대
print("Original size:", x_train.shape)
x_train = x_train.repeat(2, axis=1).repeat(2, axis=2)
x_test = x_test.repeat(2, axis=1).repeat(2, axis=2)
print("Input size:", x_train.shape)

n_image = 25
rand_idx = np.random.randint(0, len(x_train), n_image)
plt.figure(figsize=(10,10))  # 이미지의 표시 크기
for i in range(n_image):
    cifar_img=plt.subplot(5,5,i+1)
    plt.imshow(x_train[rand_idx[i]])
    label = cifar10_labels[t_train[rand_idx[i]]]
    plt.title(label)
    plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)  # 라벨과 눈금을 비표시로

# 정답을 0 또는 1로
t_train[t_train==t1] = 0  # frog
t_train[t_train==t2] = 1  # ship
t_test[t_test==t1] = 0  # frog
t_test[t_test==t2] = 1  # ship

리스트12.16 모델의 구축

In [ ]:
model = Sequential()
model.add(model_vgg16)

model.add(Flatten())  # 1차원 배열로 변환
model.add(Dense(n_mid))
model.add(Activation("relu"))
model.add(Dropout(0.5))  # 드롭 아웃
model.add(Dense(1))
model.add(Activation("sigmoid"))

# 다음에 코드를 추가한다
# ---------------- 여기부터 ----------------




# ---------------- 여기까지 ----------------

model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

리스트12.17 학습

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

x_train = x_train / 255  # 0에서 1의 범위에 넣는다
x_test = x_test / 255

# 데이터 확장
generator = ImageDataGenerator(
           rotation_range=0.2,
           width_shift_range=0.2,
           height_shift_range=0.2,
           shear_range=10,
           zoom_range=0.2,
           horizontal_flip=True)
generator.fit(x_train)

# 훈련
history = model.fit_generator(generator.flow(x_train, t_train, batch_size=batch_size),
                              epochs=epochs,
                              validation_data=(x_test, t_test))

리스트12.18 학습의 추이 확인

In [ ]:
import matplotlib.pyplot as plt

train_loss = history.history['loss']  # 훈련용 데이터의 오차
train_acc = history.history['accuracy']  # 훈련용 데이터의 정밀도
val_loss = history.history['val_loss']  # 검증용 데이터의 오차
val_acc = history.history['val_accuracy']  # 검증용 데이터의 정밀도

plt.plot(np.arange(len(train_loss)), train_loss, label='loss')
plt.plot(np.arange(len(val_loss)), val_loss, label='val_loss')
plt.legend()
plt.show()

plt.plot(np.arange(len(train_acc)), train_acc, label='acc')
plt.plot(np.arange(len(val_acc)), val_acc, label='val_acc')
plt.legend()
plt.show()

# 12.5 정답 예
  

리스트12.19 전이 학습의 정답 예

In [ ]:
model = Sequential()
model.add(model_vgg16)

model.add(Flatten())  # 1차원의 배열로 변환
model.add(Dense(n_mid))
model.add(Activation("relu"))
model.add(Dropout(0.5))  # 드롭 아웃
model.add(Dense(1))
model.add(Activation("sigmoid"))

# 다음에 코드를 추가한다
# ---------------- 여기부터 ----------------
model_vgg16.trainable = False
# ---------------- 여기까지 ----------------

model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

리스트12.20 파인 튜닝의 정답 예

In [ ]:
model = Sequential()
model.add(model_vgg16)

model.add(Flatten())  # 1차원의 배열로 변환
model.add(Dense(n_mid))
model.add(Activation("relu"))
model.add(Dropout(0.5))  # 드롭 아웃
model.add(Dense(1))
model.add(Activation("sigmoid"))

# 다음에 코드를 추가한다
# ---------------- 여기부터 ----------------
for layer in model_vgg16.layers:
    if layer.name.startswith("block5_conv"):
        layer.trainable = True
    else:
        layer.trainable = False
# ---------------- 여기까지 ----------------

model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])
model.summary()